In [122]:
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [123]:
SEQ_LENGTH = 9

all_chars = string.ascii_uppercase

def generate_palindromic_strings(length, count):
    """
    Generate palindromic strings of length `length`.
    """
    assert length % 2 == 1, "length must be odd"
    assert length >= 3, "length must be at least 3"
    strings = []
    while len(strings) < count:
        s = ""
        for j in range(length // 2):
            s += np.random.choice(list(all_chars))
        s += np.random.choice(list(all_chars))
        s += s[:length // 2][::-1]
        if s not in strings:
            strings.append(s)
    return strings

def generate_nonpalindromic_strings(length, count):
    """
    Generate non-palindromic strings of length `length`.
    """
    assert length % 2 == 1, "length must be odd"
    assert length >= 3, "length must be at least 3"
    strings = []
    while len(strings) < count:
        s = ""
        for j in range(length):
            s += np.random.choice(list(all_chars))
        if s not in strings:
            strings.append(s)
    return strings

def generate_nearly_palindromic_strings(length, count):
    """
    Generate nearly palindromic strings of length `length`.
    """
    assert length % 2 == 1, "length must be odd"
    assert length >= 3, "length must be at least 3"
    palindromic_strings = generate_palindromic_strings(length, count)
    strings = []
    for s in palindromic_strings:
        s = list(s)
        i = np.random.randint(0, length // 2)
        j = np.random.randint(0, length // 2)
        tries = 0
        while s[i] == s[j] and tries < 10:
            tries += 1
            j = np.random.randint(0, length // 2)
        if tries == 10:
            continue
        s[i], s[j] = s[j], s[i]
        strings.append("".join(s))
    return palindromic_strings + strings

In [124]:
# generate 5000 palindromic strings 
palindromic_strings = generate_palindromic_strings(SEQ_LENGTH, 5000)
# generate 5000 non-palindromic strings 
nonpalindromic_strings = generate_nonpalindromic_strings(SEQ_LENGTH, 5000)
# generate 10000 challenging strings
challenge_strings = generate_nearly_palindromic_strings(SEQ_LENGTH, 10000)

In [125]:
def is_palindrome(s):
    """
    Returns True if s is a palindrome, False otherwise.
    """
    return s == s[::-1]

In [151]:
for s in palindromic_strings:
    assert is_palindrome(s)
for s in nonpalindromic_strings:
    assert not is_palindrome(s)
palindrome_count = 0
for s in challenge_strings:
    if is_palindrome(s):
        palindrome_count += 1
palindrome_count

10000

In [127]:
dataset = palindromic_strings + nonpalindromic_strings
labels = [1] * 5000 + [0] * 5000
challenge_dataset = challenge_strings
challenge_labels = [1] * 5000 + [0] * (len(challenge_strings) - 5000)
# train test split
train_dataset, test_dataset, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.2, random_state=42, stratify=labels)

In [128]:
len(train_dataset), len(test_dataset), len(challenge_dataset), len(train_labels), len(test_labels), len(challenge_labels)

(8000, 2000, 19999, 8000, 2000, 19999)

# Count Vectorizer Based

In [129]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [130]:
X_train_counts = count_vect.fit_transform(train_dataset)
X_test_counts = count_vect.transform(test_dataset)
X_challenge_counts = count_vect.transform(challenge_dataset)

In [131]:
X_train_counts.shape, X_test_counts.shape

((8000, 8000), (2000, 8000))

In [132]:
# Train a neural network classifier
clf_count_vec = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42, max_iter=1000, verbose=True, early_stopping=True, validation_fraction=0.1)
clf_count_vec.fit(X_train_counts, train_labels)

Iteration 1, loss = 0.69330099
Validation score: 0.498750
Iteration 2, loss = 0.66677961
Validation score: 0.513750
Iteration 3, loss = 0.48777775
Validation score: 0.505000
Iteration 4, loss = 0.13240191
Validation score: 0.496250
Iteration 5, loss = 0.01782660
Validation score: 0.498750
Iteration 6, loss = 0.00640700
Validation score: 0.497500
Iteration 7, loss = 0.00377833
Validation score: 0.500000
Iteration 8, loss = 0.00259763
Validation score: 0.500000
Iteration 9, loss = 0.00193143
Validation score: 0.500000
Iteration 10, loss = 0.00151332
Validation score: 0.500000
Iteration 11, loss = 0.00123351
Validation score: 0.500000
Iteration 12, loss = 0.00103603
Validation score: 0.500000
Iteration 13, loss = 0.00089140
Validation score: 0.500000
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100), max_iter=1000,
              random_state=42, verbose=True)

In [133]:
test_preds = clf_count_vec.predict(X_test_counts)
classification_report(test_labels, test_preds, output_dict=True)

d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1000.0},
 '1': {'precision': 0.5,
  'recall': 1.0,
  'f1-score': 0.6666666666666666,
  'support': 1000.0},
 'accuracy': 0.5,
 'macro avg': {'precision': 0.25,
  'recall': 0.5,
  'f1-score': 0.3333333333333333,
  'support': 2000.0},
 'weighted avg': {'precision': 0.25,
  'recall': 0.5,
  'f1-score': 0.3333333333333333,
  'support': 2000.0}}

In [134]:
challenge_preds = clf_count_vec.predict(X_challenge_counts)
classification_report(challenge_labels, challenge_preds, output_dict=True)

d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 14999.0},
 '1': {'precision': 0.2500125006250313,
  'recall': 1.0,
  'f1-score': 0.4000160006400256,
  'support': 5000.0},
 'accuracy': 0.2500125006250313,
 'macro avg': {'precision': 0.12500625031251564,
  'recall': 0.5,
  'f1-score': 0.2000080003200128,
  'support': 19999.0},
 'weighted avg': {'precision': 0.06250625046878126,
  'recall': 0.2500125006250313,
  'f1-score': 0.10000900061003691,
  'support': 19999.0}}

# TF-IDF Vectorizer

In [136]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
X_challenge_tfidf = tfidf_transformer.transform(X_challenge_counts)

In [137]:
X_train_tfidf.shape, X_test_tfidf.shape

((8000, 8000), (2000, 8000))

In [138]:
tf_idf_clf = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42, max_iter=1000, verbose=True, early_stopping=True, validation_fraction=0.1)
tf_idf_clf.fit(X_train_tfidf, train_labels)

Iteration 1, loss = 0.69330099
Validation score: 0.498750
Iteration 2, loss = 0.66677961
Validation score: 0.513750
Iteration 3, loss = 0.48777775
Validation score: 0.505000
Iteration 4, loss = 0.13240191
Validation score: 0.496250
Iteration 5, loss = 0.01782660
Validation score: 0.498750
Iteration 6, loss = 0.00640700
Validation score: 0.497500
Iteration 7, loss = 0.00377833
Validation score: 0.500000
Iteration 8, loss = 0.00259763
Validation score: 0.500000
Iteration 9, loss = 0.00193143
Validation score: 0.500000
Iteration 10, loss = 0.00151332
Validation score: 0.500000
Iteration 11, loss = 0.00123351
Validation score: 0.500000
Iteration 12, loss = 0.00103603
Validation score: 0.500000
Iteration 13, loss = 0.00089140
Validation score: 0.500000
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100), max_iter=1000,
              random_state=42, verbose=True)

In [139]:
test_preds = tf_idf_clf.predict(X_test_tfidf)
confusion_matrix(test_labels, test_preds)

array([[   0, 1000],
       [   0, 1000]], dtype=int64)

In [140]:
challenge_preds = tf_idf_clf.predict(X_challenge_tfidf)
classification_report(challenge_labels, challenge_preds, output_dict=True)

d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 14999.0},
 '1': {'precision': 0.2500125006250313,
  'recall': 1.0,
  'f1-score': 0.4000160006400256,
  'support': 5000.0},
 'accuracy': 0.2500125006250313,
 'macro avg': {'precision': 0.12500625031251564,
  'recall': 0.5,
  'f1-score': 0.2000080003200128,
  'support': 19999.0},
 'weighted avg': {'precision': 0.06250625046878126,
  'recall': 0.2500125006250313,
  'f1-score': 0.10000900061003691,
  'support': 19999.0}}

# Encode Using Alphabet Arrays With Bools

In [141]:
def encode(seq):
    seq = seq.upper()
    assert len(seq) == SEQ_LENGTH, "length of sequence must be {}".format(SEQ_LENGTH)
    encoding = [0]*len(all_chars)
    for i, char in enumerate(seq):
        encoding[all_chars.index(char)] = 1
    return encoding

encoded_train_dataset = [encode(seq) for seq in train_dataset]
encoded_test_dataset = [encode(seq) for seq in test_dataset]
encoded_challenge_dataset = [encode(seq) for seq in challenge_dataset]

In [142]:
clf = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42, max_iter=1000, verbose=True, early_stopping=True, validation_fraction=0.1)
clf.fit(encoded_train_dataset, train_labels)

Iteration 1, loss = 0.59960631
Validation score: 0.841250
Iteration 2, loss = 0.35448123
Validation score: 0.975000
Iteration 3, loss = 0.13834979
Validation score: 0.981250
Iteration 4, loss = 0.07248609
Validation score: 0.985000
Iteration 5, loss = 0.05298355
Validation score: 0.981250
Iteration 6, loss = 0.04309932
Validation score: 0.990000
Iteration 7, loss = 0.03661357
Validation score: 0.990000
Iteration 8, loss = 0.03214211
Validation score: 0.990000
Iteration 9, loss = 0.02937653
Validation score: 0.990000
Iteration 10, loss = 0.02605136
Validation score: 0.992500
Iteration 11, loss = 0.02478028
Validation score: 0.991250
Iteration 12, loss = 0.02218605
Validation score: 0.988750
Iteration 13, loss = 0.02070923
Validation score: 0.991250
Iteration 14, loss = 0.01956233
Validation score: 0.991250
Iteration 15, loss = 0.01783790
Validation score: 0.990000
Iteration 16, loss = 0.01796825
Validation score: 0.991250
Iteration 17, loss = 0.01510266
Validation score: 0.991250
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100), max_iter=1000,
              random_state=42, verbose=True)

In [143]:
test_preds = clf.predict(encoded_test_dataset)
classification_report(test_labels, test_preds, output_dict=True)

{'0': {'precision': 1.0,
  'recall': 0.99,
  'f1-score': 0.9949748743718593,
  'support': 1000.0},
 '1': {'precision': 0.9900990099009901,
  'recall': 1.0,
  'f1-score': 0.9950248756218906,
  'support': 1000.0},
 'accuracy': 0.995,
 'macro avg': {'precision': 0.995049504950495,
  'recall': 0.995,
  'f1-score': 0.9949998749968749,
  'support': 2000.0},
 'weighted avg': {'precision': 0.995049504950495,
  'recall': 0.995,
  'f1-score': 0.9949998749968749,
  'support': 2000.0}}

In [144]:
challenge_preds = clf.predict(encoded_challenge_dataset)
classification_report(challenge_labels, challenge_preds, output_dict=True)

d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 14999.0},
 '1': {'precision': 0.2500125006250313,
  'recall': 1.0,
  'f1-score': 0.4000160006400256,
  'support': 5000.0},
 'accuracy': 0.2500125006250313,
 'macro avg': {'precision': 0.12500625031251564,
  'recall': 0.5,
  'f1-score': 0.2000080003200128,
  'support': 19999.0},
 'weighted avg': {'precision': 0.06250625046878126,
  'recall': 0.2500125006250313,
  'f1-score': 0.10000900061003691,
  'support': 19999.0}}

# Encode Using Alphabet Arrays With Frequencies

In [145]:
def encode_with_freq(seq):
    seq = seq.upper()
    assert len(seq) == SEQ_LENGTH, "length of sequence must be {}".format(SEQ_LENGTH)
    encoding = [0]*len(all_chars)
    for i, char in enumerate(seq):
        encoding[all_chars.index(char)] += 1
    return encoding

encoded_with_freq_train_dataset = [encode_with_freq(seq) for seq in train_dataset]
encoded_with_freq_test_dataset = [encode_with_freq(seq) for seq in test_dataset]
encoded_with_freq_challenge_dataset = [encode_with_freq(seq) for seq in challenge_dataset]

In [146]:
clf_encoded_with_freq = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42, max_iter=1000, verbose=True, early_stopping=True, validation_fraction=0.1)
clf_encoded_with_freq.fit(encoded_with_freq_train_dataset, train_labels)

Iteration 1, loss = 0.64408131
Validation score: 0.778750
Iteration 2, loss = 0.49354465
Validation score: 0.838750
Iteration 3, loss = 0.35086734
Validation score: 0.863750
Iteration 4, loss = 0.27638589
Validation score: 0.890000
Iteration 5, loss = 0.23516032
Validation score: 0.906250
Iteration 6, loss = 0.20770107
Validation score: 0.917500
Iteration 7, loss = 0.18468332
Validation score: 0.926250
Iteration 8, loss = 0.16777078
Validation score: 0.912500
Iteration 9, loss = 0.15614554
Validation score: 0.930000
Iteration 10, loss = 0.14171770
Validation score: 0.911250
Iteration 11, loss = 0.13325240
Validation score: 0.935000
Iteration 12, loss = 0.12459984
Validation score: 0.937500
Iteration 13, loss = 0.11839305
Validation score: 0.938750
Iteration 14, loss = 0.11066509
Validation score: 0.935000
Iteration 15, loss = 0.10635911
Validation score: 0.940000
Iteration 16, loss = 0.11025128
Validation score: 0.933750
Iteration 17, loss = 0.09828306
Validation score: 0.935000
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100), max_iter=1000,
              random_state=42, verbose=True)

In [149]:
test_preds = clf_encoded_with_freq.predict(encoded_with_freq_test_dataset)
classification_report(test_labels, test_preds, output_dict=True)

{'0': {'precision': 0.9439252336448598,
  'recall': 0.909,
  'f1-score': 0.9261334691798268,
  'support': 1000.0},
 '1': {'precision': 0.9122468659594986,
  'recall': 0.946,
  'f1-score': 0.9288168875797741,
  'support': 1000.0},
 'accuracy': 0.9275,
 'macro avg': {'precision': 0.9280860498021792,
  'recall': 0.9275,
  'f1-score': 0.9274751783798004,
  'support': 2000.0},
 'weighted avg': {'precision': 0.9280860498021791,
  'recall': 0.9275,
  'f1-score': 0.9274751783798003,
  'support': 2000.0}}

In [148]:
challenge_preds = clf_encoded_with_freq.predict(encoded_with_freq_challenge_dataset)
classification_report(challenge_labels, challenge_preds, output_dict=True)

{'0': {'precision': 0.7617586912065439,
  'recall': 0.04966997799853323,
  'f1-score': 0.09325905989860424,
  'support': 14999.0},
 '1': {'precision': 0.25061773828925926,
  'recall': 0.9534,
  'f1-score': 0.3969027101286374,
  'support': 5000.0},
 'accuracy': 0.27561378068903447,
 'macro avg': {'precision': 0.5061882147479015,
  'recall': 0.5015349889992666,
  'f1-score': 0.24508088501362083,
  'support': 19999.0},
 'weighted avg': {'precision': 0.6339670633958323,
  'recall': 0.27561378068903447,
  'f1-score': 0.16917376819152719,
  'support': 19999.0}}